In [ ]:
import requests
import pandas as pd

def fetch_helm_charts_with_enhancements():
    url = "https://artifacthub.io/api/v1/packages/search"
    charts = []
    offset = 0
    limit = 60  
    while True:
        response = requests.get(url, params={"limit": limit, "offset": offset})
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break

        data = response.json()
        if not data['packages']: 
            break

        for chart in data['packages']:
           
            charts.append({
                "chart_name": chart.get("name", "Unknown"),
                "description": chart.get("description", "No description available"),
                "category": chart.get("category", "Unknown"),
                "tags": ",".join(chart.get("keywords", [])),
                "maintainers": ",".join([m.get("name", "Unknown") for m in chart.get("maintainers", [])]),
                "popularity": chart.get("stars", 0),
                "provider": chart.get("repository", {}).get("provider", "Unknown"),
                "dependencies": ",".join(chart.get("dependencies", [])),
                "version": chart.get("version", "Unknown"),
                "verified_publisher": chart.get("repository", {}).get("verified_publisher", False),
                "last_updated": chart.get("created_at", "Unknown"),
                "developer": chart.get("repository", {}).get("name", "Unknown"),
                "repository_url": chart.get("repository", {}).get("url", "Unknown"),
                "license_type": chart.get("license", "Unknown"),
                "average_rating": chart.get("rating", {}).get("average", None),
                "rating_count": chart.get("rating", {}).get("count", 0),
                "installation_size": chart.get("install_size", "Unknown"),
            })

        offset += limit  

    return pd.DataFrame(charts)


charts_df = fetch_helm_charts_with_enhancements()
charts_df.to_csv("helm_charts_enhanced.csv", index=False)
print(f"Saved {len(charts_df)} charts to helm_charts_enhanced.csv")


Failed to fetch data: 429
Saved 5820 charts to helm_charts_enhanced.csv


In [ ]:
# Check for missing values
missing_summary = charts_df.isnull().sum()
print("Missing Values Summary:")
print(missing_summary)

# Replace "Unknown" and completely empty values with NaN for easier handling
charts_df.replace(['Unknown', 'No description available', ''], pd.NA, inplace=True)


Missing Values Summary:
chart_name               0
description              0
category                 0
tags                     0
maintainers              0
popularity               0
provider                 0
dependencies             0
version                  0
verified_publisher       0
last_updated             0
developer                0
repository_url           0
license_type             0
average_rating        5820
rating_count             0
installation_size        0
dtype: int64


In [ ]:
# mapping for categories
category_mapping = {
    1: "Monitoring",
    2: "Database",
    3: "Secrets Management",
    4: "Visualization",
    5: "Ingress Controller",
    6: "Certificate Management"
}
# Map and fill missing categories
charts_df['category'] = charts_df['category'].map(category_mapping).fillna('Uncategorized')


In [ ]:
charts_df

,chart_name,description,category,tags,maintainers,popularity,provider,dependencies,version,verified_publisher,last_updated,developer,repository_url,license_type,average_rating,rating_count,installation_size
0,kube-prometheus-stack,kube-prometheus-stack collects Kubernetes mani...,Visualization,<NA>,<NA>,937,<NA>,<NA>,68.1.1,True,<NA>,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,None,0,<NA>
1,cert-manager,A Helm chart for cert-manager,Certificate Management,<NA>,<NA>,740,<NA>,<NA>,1.16.3,True,<NA>,cert-manager,https://charts.jetstack.io,Apache-2.0,None,0,<NA>
2,ingress-nginx,Ingress controller for Kubernetes using NGINX ...,Ingress Controller,<NA>,<NA>,703,<NA>,<NA>,4.12.0,False,<NA>,ingress-nginx,https://kubernetes.github.io/ingress-nginx,<NA>,None,0,<NA>
3,argo-cd,"A Helm chart for Argo CD, a declarative, GitOp...",Secrets Management,<NA>,<NA>,622,<NA>,<NA>,7.7.16,True,<NA>,argo,https://argoproj.github.io/argo-helm,<NA>,None,0,<NA>
4,prometheus,Prometheus is a monitoring system and time ser...,Visualization,<NA>,<NA>,445,<NA>,<NA>,26.1.0,True,<NA>,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,None,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,kube-prometheus-stack-crd,Custom Resource Definitions for kubernetes pro...,Uncategorized,<NA>,<NA>,0,<NA>,<NA>,49.0.0,True,<NA>,alekc,https://charts.alekc.dev,<NA>,None,0,<NA>
5816,kube-prometheus-stack-crds,Manage kube-prometheus-stack CRDs,Visualization,<NA>,<NA>,0,<NA>,<NA>,40.0.0,True,<NA>,wiremind,https://wiremind.github.io/wiremind-helm-charts,<NA>,None,0,<NA>
5817,kube-proxy,Helm chart for managing kube-proxy.,Ingress Controller,<NA>,<NA>,0,<NA>,<NA>,0.0.6,True,<NA>,stevehipwell,https://stevehipwell.github.io/helm-charts/,<NA>,None,0,<NA>
5818,kuberest,Allow to Control remote REST api endpoints fro...,Uncategorized,<NA>,<NA>,0,<NA>,<NA>,1.3.0,True,<NA>,kuberest,https://sebt3.github.io/kuberest/,<NA>,None,0,<NA>


In [ ]:
# Combine relevant fields into tags
def generate_tags(row):
    tags = []

    # Add existing tags if available
    if pd.notna(row['tags']):
        tags.extend(row['tags'].split(', '))

    # Add category
    if pd.notna(row['category']):
        tags.append(row['category'].lower())

    # Add developer name
    if pd.notna(row['developer']):
        tags.append(row['developer'].lower().replace(' ', '-'))

    # Add repository keywords
    if pd.notna(row['repository_url']):
        repo_keywords = row['repository_url'].split('/')[-2:]
        tags.extend([kw.lower().replace('-', '_') for kw in repo_keywords])

    # Add key terms from description
    if pd.notna(row['description']):
        description_keywords = row['description'].split()[:3]  # Take first 3 words
        tags.extend([kw.lower().strip(',') for kw in description_keywords])

    # Remove duplicates
    return ', '.join(set(tags))

# Apply tag generation
charts_df['tags'] = charts_df.apply(generate_tags, axis=1)


In [ ]:
charts_df

,chart_name,description,category,tags,maintainers,popularity,provider,dependencies,version,verified_publisher,last_updated,developer,repository_url,license_type,average_rating,rating_count,installation_size
0,kube-prometheus-stack,kube-prometheus-stack collects Kubernetes mani...,Visualization,"kube-prometheus-stack, prometheus_community.gi...",<NA>,937,<NA>,<NA>,68.1.1,True,<NA>,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,None,0,<NA>
1,cert-manager,A Helm chart for cert-manager,Certificate Management,", helm, a, chart, charts.jetstack.io, cert-man...",<NA>,740,<NA>,<NA>,1.16.3,True,<NA>,cert-manager,https://charts.jetstack.io,Apache-2.0,None,0,<NA>
2,ingress-nginx,Ingress controller for Kubernetes using NGINX ...,Ingress Controller,"ingress-nginx, ingress controller, ingress, co...",<NA>,703,<NA>,<NA>,4.12.0,False,<NA>,ingress-nginx,https://kubernetes.github.io/ingress-nginx,<NA>,None,0,<NA>
3,argo-cd,"A Helm chart for Argo CD, a declarative, GitOp...",Secrets Management,"argo, argo_helm, secrets management, helm, a, ...",<NA>,622,<NA>,<NA>,7.7.16,True,<NA>,argo,https://argoproj.github.io/argo-helm,<NA>,None,0,<NA>
4,prometheus,Prometheus is a monitoring system and time ser...,Visualization,", is, prometheus, prometheus_community.github....",<NA>,445,<NA>,<NA>,26.1.0,True,<NA>,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,None,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,kube-prometheus-stack-crd,Custom Resource Definitions for kubernetes pro...,Uncategorized,", alekc, resource, charts.alekc.dev, uncategor...",<NA>,0,<NA>,<NA>,49.0.0,True,<NA>,alekc,https://charts.alekc.dev,<NA>,None,0,<NA>
5816,kube-prometheus-stack-crds,Manage kube-prometheus-stack CRDs,Visualization,", wiremind_helm_charts, wiremind.github.io, cr...",<NA>,0,<NA>,<NA>,40.0.0,True,<NA>,wiremind,https://wiremind.github.io/wiremind-helm-charts,<NA>,None,0,<NA>
5817,kube-proxy,Helm chart for managing kube-proxy.,Ingress Controller,", stevehipwell, ingress controller, helm, helm...",<NA>,0,<NA>,<NA>,0.0.6,True,<NA>,stevehipwell,https://stevehipwell.github.io/helm-charts/,<NA>,None,0,<NA>
5818,kuberest,Allow to Control remote REST api endpoints fro...,Uncategorized,", to, control, cluster, uncategorized, kuberes...",<NA>,0,<NA>,<NA>,1.3.0,True,<NA>,kuberest,https://sebt3.github.io/kuberest/,<NA>,None,0,<NA>


In [ ]:
charts_df.to_csv("m", index=False)

In [ ]:
# Fill missing maintainers and dependencies
charts_df['maintainers'] = charts_df['maintainers'].fillna('Not Specified')
charts_df['dependencies'] = charts_df['dependencies'].fillna('Not Specified')


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Scale numeric fields
scaler = MinMaxScaler()
charts_df['popularity_scaled'] = scaler.fit_transform(charts_df[['popularity']])
charts_df['rating_count_scaled'] = scaler.fit_transform(charts_df[['rating_count']])


In [ ]:
# Convert last_updated to datetime
charts_df['last_updated'] = pd.to_datetime(charts_df['last_updated'], errors='coerce')

# Fill missing dates with a default value (e.g., oldest date in the dataset)
earliest_date = charts_df['last_updated'].min()
charts_df['last_updated'].fillna(earliest_date, inplace=True)


<ipython-input-17-63c938ac23fa>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  charts_df['last_updated'].fillna(earliest_date, inplace=True)


In [ ]:
# Convert boolean fields
charts_df['verified_publisher'] = charts_df['verified_publisher'].astype(bool)

# Validate types
print(charts_df.dtypes)


chart_name                     object
description                    object
category                       object
tags                           object
maintainers                    object
popularity                      int64
provider                       object
dependencies                   object
version                        object
verified_publisher               bool
last_updated           datetime64[ns]
developer                      object
repository_url                 object
license_type                   object
average_rating                 object
rating_count                    int64
installation_size              object
popularity_scaled             float64
rating_count_scaled           float64
dtype: object


In [ ]:
# Preview cleaned dataset
print(charts_df.head())


              chart_name                                        description  \
0  kube-prometheus-stack  kube-prometheus-stack collects Kubernetes mani...   
1           cert-manager                      A Helm chart for cert-manager   
2          ingress-nginx  Ingress controller for Kubernetes using NGINX ...   
3                argo-cd  A Helm chart for Argo CD, a declarative, GitOp...   
4             prometheus  Prometheus is a monitoring system and time ser...   

                 category                                               tags  \
0           Visualization  kube-prometheus-stack, prometheus_community.gi...   
1  Certificate Management  , helm, a, chart, charts.jetstack.io, cert-man...   
2      Ingress Controller  ingress-nginx, ingress controller, ingress, co...   
3      Secrets Management  argo, argo_helm, secrets management, helm, a, ...   
4           Visualization  , is, prometheus, prometheus_community.github....   

     maintainers  popularity provider   depe

In [ ]:
# Check for remaining missing values
print(charts_df.isnull().sum())


chart_name                0
description              34
category                  0
tags                      0
maintainers               0
popularity                0
provider               5820
dependencies              0
version                   0
verified_publisher        0
last_updated           5820
developer                 0
repository_url            0
license_type           4631
average_rating         5820
rating_count              0
installation_size      5820
popularity_scaled         0
rating_count_scaled       0
dtype: int64


In [ ]:
charts_df.to_csv("s.csv",index=False)

In [ ]:
def refine_tags(row):
    # Split existing tags into a set for deduplication
    tags = set(row['tags'].split(', '))

    # Remove generic/common tags
    common_terms = {'helm', 'a', 'is', 'chart', 'charts', ''}
    tags = tags - common_terms

    # Add specific details from category and developer
    if pd.notna(row['category']):
        tags.add(row['category'].lower())
    if pd.notna(row['developer']):
        tags.add(row['developer'].lower().replace(' ', '_'))

    # Return as a comma-separated string
    return ', '.join(tags)

# Apply tag refinement
charts_df['tags'] = charts_df.apply(refine_tags, axis=1)


In [ ]:
# Drop entirely empty columns
columns_to_drop = ['provider', 'last_updated', 'dependencies', 'installation_size']
charts_df.drop(columns=columns_to_drop, inplace=True)

# Fill missing license_type with "Unknown"
charts_df['license_type'].fillna('Unknown', inplace=True)


<ipython-input-24-7fc9f9e33c48>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  charts_df['license_type'].fillna('Unknown', inplace=True)


In [ ]:
charts_df.to_csv("m.csv",index=False)

In [ ]:
charts_df

,chart_name,description,category,tags,maintainers,popularity,version,verified_publisher,developer,repository_url,license_type,average_rating,rating_count,popularity_scaled,rating_count_scaled
0,kube-prometheus-stack,kube-prometheus-stack collects Kubernetes mani...,Visualization,"kube-prometheus-stack, prometheus_community.gi...",Not Specified,937,68.1.1,True,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,None,0,1.000000,0.0
1,cert-manager,A Helm chart for cert-manager,Certificate Management,"cert-manager, charts.jetstack.io, certificate ...",Not Specified,740,1.16.3,True,cert-manager,https://charts.jetstack.io,Apache-2.0,None,0,0.789755,0.0
2,ingress-nginx,Ingress controller for Kubernetes using NGINX ...,Ingress Controller,"ingress-nginx, ingress controller, ingress, co...",Not Specified,703,4.12.0,False,ingress-nginx,https://kubernetes.github.io/ingress-nginx,Unknown,None,0,0.750267,0.0
3,argo-cd,"A Helm chart for Argo CD, a declarative, GitOp...",Secrets Management,"argo, argo_helm, kubernetes, argoproj.github.i...",Not Specified,622,7.7.16,True,argo,https://argoproj.github.io/argo-helm,Unknown,None,0,0.663821,0.0
4,prometheus,Prometheus is a monitoring system and time ser...,Visualization,"prometheus, prometheus_community.github.io, he...",Not Specified,445,26.1.0,True,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,None,0,0.474920,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,kube-prometheus-stack-crd,Custom Resource Definitions for kubernetes pro...,Uncategorized,"alekc, resource, charts.alekc.dev, uncategoriz...",Not Specified,0,49.0.0,True,alekc,https://charts.alekc.dev,Unknown,None,0,0.000000,0.0
5816,kube-prometheus-stack-crds,Manage kube-prometheus-stack CRDs,Visualization,"wiremind.github.io, wiremind_helm_charts, crds...",Not Specified,0,40.0.0,True,wiremind,https://wiremind.github.io/wiremind-helm-charts,Unknown,None,0,0.000000,0.0
5817,kube-proxy,Helm chart for managing kube-proxy.,Ingress Controller,"for, stevehipwell, helm_charts, ingress contro...",Not Specified,0,0.0.6,True,stevehipwell,https://stevehipwell.github.io/helm-charts/,Unknown,None,0,0.000000,0.0
5818,kuberest,Allow to Control remote REST api endpoints fro...,Uncategorized,"to, control, cluster, uncategorized, kuberest,...",Not Specified,0,1.3.0,True,kuberest,https://sebt3.github.io/kuberest/,Unknown,None,0,0.000000,0.0


In [ ]:
# Drop columns that are entirely empty or irrelevant
columns_to_drop = ['average_rating', 'maintainers']
charts_df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
# Remove generic or overly common terms from tags
common_terms = {'helm', 'chart', 'charts', 'kubernetes'}
charts_df['tags'] = charts_df['tags'].apply(
    lambda x: ', '.join(set(x.split(', ')) - common_terms)
)


In [ ]:
# Check remaining missing values
missing_summary = charts_df.isnull().sum()
print(missing_summary)


chart_name              0
description            34
category                0
tags                    0
popularity              0
version                 0
verified_publisher      0
developer               0
repository_url          0
license_type            0
rating_count            0
popularity_scaled       0
rating_count_scaled     0
dtype: int64


In [ ]:
charts_df.to_csv("l.csv",index=False)

In [ ]:


import pandas as pd

data = pd.read_csv('l.csv')

In [ ]:
data

,chart_name,description,category,tags,popularity,version,verified_publisher,developer,repository_url,license_type,rating_count,popularity_scaled,rating_count_scaled
0,kube-prometheus-stack,kube-prometheus-stack collects Kubernetes mani...,Visualization,"kube-prometheus-stack, prometheus_community.gi...",937,68.1.1,True,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,0,1.000000,0.0
1,cert-manager,A Helm chart for cert-manager,Certificate Management,"cert-manager, charts.jetstack.io, certificate ...",740,1.16.3,True,cert-manager,https://charts.jetstack.io,Apache-2.0,0,0.789755,0.0
2,ingress-nginx,Ingress controller for Kubernetes using NGINX ...,Ingress Controller,"ingress-nginx, ingress controller, ingress, co...",703,4.12.0,False,ingress-nginx,https://kubernetes.github.io/ingress-nginx,Unknown,0,0.750267,0.0
3,argo-cd,"A Helm chart for Argo CD, a declarative, GitOp...",Secrets Management,"argo, secrets management, argoproj.github.io, ...",622,7.7.16,True,argo,https://argoproj.github.io/argo-helm,Unknown,0,0.663821,0.0
4,prometheus,Prometheus is a monitoring system and time ser...,Visualization,"prometheus, prometheus_community.github.io, he...",445,26.1.0,True,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,0,0.474920,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,kube-prometheus-stack-crd,Custom Resource Definitions for kubernetes pro...,Uncategorized,"alekc, resource, charts.alekc.dev, uncategoriz...",0,49.0.0,True,alekc,https://charts.alekc.dev,Unknown,0,0.000000,0.0
5816,kube-prometheus-stack-crds,Manage kube-prometheus-stack CRDs,Visualization,"wiremind.github.io, wiremind_helm_charts, crds...",0,40.0.0,True,wiremind,https://wiremind.github.io/wiremind-helm-charts,Unknown,0,0.000000,0.0
5817,kube-proxy,Helm chart for managing kube-proxy.,Ingress Controller,"for, helm_charts, ingress controller, stevehip...",0,0.0.6,True,stevehipwell,https://stevehipwell.github.io/helm-charts/,Unknown,0,0.000000,0.0
5818,kuberest,Allow to Control remote REST api endpoints fro...,Uncategorized,"to, control, cluster, uncategorized, kuberest,...",0,1.3.0,True,kuberest,https://sebt3.github.io/kuberest/,Unknown,0,0.000000,0.0


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [ ]:
# Analyze the Dataset
data_info = data.info()
missing_values = data.isnull().sum()
unique_categories = data['category'].nunique()
unique_tags = data['tags'].nunique()
unique_developers = data['developer'].nunique()
numerical_summary = data.describe()

# Fill missing descriptions
data['description'].fillna("No description available", inplace=True)

# Normalize and Categorize Popularity
popularity_bins = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
data['popularity_category'] = popularity_bins.fit_transform(data[['popularity']]).astype(int)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5820 entries, 0 to 5819
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   chart_name           5820 non-null   object 
 1   description          5786 non-null   object 
 2   category             5820 non-null   object 
 3   tags                 5820 non-null   object 
 4   popularity           5820 non-null   int64  
 5   version              5820 non-null   object 
 6   verified_publisher   5820 non-null   bool   
 7   developer            5820 non-null   object 
 8   repository_url       5820 non-null   object 
 9   license_type         5820 non-null   object 
 10  rating_count         5820 non-null   int64  
 11  popularity_scaled    5820 non-null   float64
 12  rating_count_scaled  5820 non-null   float64
dtypes: bool(1), float64(2), int64(2), object(8)
memory usage: 551.4+ KB


<ipython-input-4-f4926af07eea>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['description'].fillna("No description available", inplace=True)


In [ ]:
# Topic Modeling on Descriptions and Tags
data['text_combined'] = data['description'] + ' ' + data['tags']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text_combined'])

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(tfidf_matrix)

def display_topics(model, feature_names, n_top_words):
    topics = {}
    for idx, topic in enumerate(model.components_):
        topics[f"Topic {idx+1}"] = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    return topics

topics = display_topics(lda, tfidf_vectorizer.get_feature_names_out(), 10)

# Categorize Uncategorized Charts
uncategorized_data = data[data['category'] == 'Uncategorized']
uncategorized_topics = lda.transform(tfidf_vectorizer.transform(uncategorized_data['text_combined']))

In [ ]:
category_map = {
    0: "Operator Management",
    1: "Ingress and Controller Tools",
    2: "Monitoring and Visualization",
    3: "General Helm Tools",
    4: "Policy and Governance",
}
uncategorized_data['predicted_category'] = uncategorized_topics.argmax(axis=1)
uncategorized_data['predicted_category'] = uncategorized_data['predicted_category'].map(category_map)
data.loc[data['category'] == 'Uncategorized', 'category'] = uncategorized_data['predicted_category']

# Add Topic Probabilities as Features
topic_columns = [f"topic_{i+1}" for i in range(lda.components_.shape[0])]
topic_probabilities = lda.transform(tfidf_matrix)
topic_features = pd.DataFrame(topic_probabilities, columns=topic_columns)
data_with_topics = pd.concat([data.reset_index(drop=True), topic_features], axis=1)

<ipython-input-6-617237272354>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uncategorized_data['predicted_category'] = uncategorized_topics.argmax(axis=1)
<ipython-input-6-617237272354>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uncategorized_data['predicted_category'] = uncategorized_data['predicted_category'].map(category_map)


In [ ]:
data_with_topics
data_with_topics.to_csv('Helm_Chart_Dataset_with_Categories_and_Topics1.csv', index=False)

,chart_name,description,category,tags,popularity,version,verified_publisher,developer,repository_url,license_type,rating_count,popularity_scaled,rating_count_scaled,popularity_category,text_combined,topic_1,topic_2,topic_3,topic_4,topic_5
0,kube-prometheus-stack,kube-prometheus-stack collects Kubernetes mani...,Visualization,"kube-prometheus-stack, prometheus_community.gi...",937,68.1.1,True,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,0,1.000000,0.0,2,kube-prometheus-stack collects Kubernetes mani...,0.042010,0.041977,0.834874,0.040617,0.040523
1,cert-manager,A Helm chart for cert-manager,Certificate Management,"cert-manager, charts.jetstack.io, certificate ...",740,1.16.3,True,cert-manager,https://charts.jetstack.io,Apache-2.0,0,0.789755,0.0,2,"A Helm chart for cert-manager cert-manager, ch...",0.058456,0.058239,0.766482,0.058793,0.058030
2,ingress-nginx,Ingress controller for Kubernetes using NGINX ...,Ingress Controller,"ingress-nginx, ingress controller, ingress, co...",703,4.12.0,False,ingress-nginx,https://kubernetes.github.io/ingress-nginx,Unknown,0,0.750267,0.0,2,Ingress controller for Kubernetes using NGINX ...,0.051561,0.794434,0.051289,0.051513,0.051203
3,argo-cd,"A Helm chart for Argo CD, a declarative, GitOp...",Secrets Management,"argo, secrets management, argoproj.github.io, ...",622,7.7.16,True,argo,https://argoproj.github.io/argo-helm,Unknown,0,0.663821,0.0,1,"A Helm chart for Argo CD, a declarative, GitOp...",0.808958,0.047761,0.047651,0.048343,0.047286
4,prometheus,Prometheus is a monitoring system and time ser...,Visualization,"prometheus, prometheus_community.github.io, he...",445,26.1.0,True,prometheus-community,https://prometheus-community.github.io/helm-ch...,Apache-2.0,0,0.474920,0.0,1,Prometheus is a monitoring system and time ser...,0.053766,0.053567,0.786787,0.053639,0.052241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,kube-prometheus-stack-crd,Custom Resource Definitions for kubernetes pro...,Operator Management,"alekc, resource, charts.alekc.dev, uncategoriz...",0,49.0.0,True,alekc,https://charts.alekc.dev,Unknown,0,0.000000,0.0,0,Custom Resource Definitions for kubernetes pro...,0.423740,0.055827,0.407986,0.055199,0.057248
5816,kube-prometheus-stack-crds,Manage kube-prometheus-stack CRDs,Visualization,"wiremind.github.io, wiremind_helm_charts, crds...",0,40.0.0,True,wiremind,https://wiremind.github.io/wiremind-helm-charts,Unknown,0,0.000000,0.0,0,Manage kube-prometheus-stack CRDs wiremind.git...,0.052505,0.052484,0.790901,0.052127,0.051984
5817,kube-proxy,Helm chart for managing kube-proxy.,Ingress Controller,"for, helm_charts, ingress controller, stevehip...",0,0.0.6,True,stevehipwell,https://stevehipwell.github.io/helm-charts/,Unknown,0,0.000000,0.0,0,"Helm chart for managing kube-proxy. for, helm_...",0.054814,0.463629,0.374580,0.053997,0.052979
5818,kuberest,Allow to Control remote REST api endpoints fro...,Policy and Governance,"to, control, cluster, uncategorized, kuberest,...",0,1.3.0,True,kuberest,https://sebt3.github.io/kuberest/,Unknown,0,0.000000,0.0,0,Allow to Control remote REST api endpoints fro...,0.061654,0.060115,0.059341,0.060832,0.758057
